In [1]:
import llm_core.llm as L
from srs.SRS import SRS
import re
import json

In [2]:
#initialize
llm1 = L.LLM("You are a grader for beginner language learning students. "
            "You judge their response for grammar and "
            "concisely describe how each word is correct or incorrect in the sentence.")
llm2 = L.LLM("""You are a grader for beginner language learning students.
                You judge their response for grammar and correctness and grade accordingly.
                Each mistake has a short description with it for what they did wrong.
                If there is nothing wrong, simply don't say anything.""")
srs = SRS()


Initializing Global LLM Instance


In [3]:
#mistakes in first and last sentence
response = "Io vado a la scuola. Oggi è una bella giornata. Lei mangia molto di pizza."

# simply grade each word in the context of the sentence
def grade_response(response : str) -> list:
    output = list()
    sentences = [sentence for sentence in response.split('.') if sentence != '']
    for sentence in sentences:
        words = sentence.split(' ')
        for word in words:
            llm1.reset()
            grade1 = llm1(
                f"Evaluate {word} for grammatical correctness in the sentence: {sentence}",
                response_format={
                    'reason': str, 'Correct? yes or no?' : str
                },
                max_tokens=1000,
                temperature=0,
                verbose=False
            )
            output.append(grade1)
    return output

# remind system with what it's already graded in the sentence
def grade_response2(response : str) -> list:
    output = list()
    sentences = [sentence for sentence in response.split('.') if sentence != '']
    for sentence in sentences:
        grades = ''
        words = sentence.split(' ')
        for word in words:
            llm1.reset()
            grade1 = llm1(
                grades_prompt(grades) + 
                f"Evaluate {word} for grammatical correctness in the sentence: {sentence}",
                response_format={
                    'reason': str, 'Correct? yes or no?' : str
                },
                max_tokens=None,
                temperature=0,
                verbose=False
            )
            output.append(grade1)
            grades = str(output)
    return output

def grades_prompt(grades : str) -> str:
    if (grades != ''):
        grades = "You have evaluated the sentence so far as follows: " + grades
    return grades

print(json.dumps(grade_response(response), indent=4))
print('####################################')
print(json.dumps(grade_response2(response), indent=4))

[
    {
        "reason": "The word 'Io' is grammatically correct as it is used as the subject pronoun in this sentence, which is equivalent to 'I' in English.",
        "Correct? yes or no?": "yes"
    },
    {
        "reason": "The word 'vado' is grammatically correct in the sentence. It is the first person singular, present tense of the Italian verb 'andare' which means 'to go'. In this sentence, it is used correctly to indicate that the speaker is going to school.",
        "Correct? yes or no?": "yes"
    },
    {
        "reason": "The word 'a' is a preposition in English, but in this Italian sentence, 'a' is incorrect. The correct preposition to use here is 'a', which means 'to' in English. However, it should be noted that 'a' should be joined with the next word without a space, as is the convention in Italian. So, the correct sentence is 'Io vado a scuola'.",
        "Correct? yes or no?": "no, it is incorrect in this context."
    },
    {
        "reason": "In the sentence '